In [10]:
import pandas as pd
import numpy as np
import torch
import sklearn.metrics as metrics
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset, TensorDataset, SubsetRandomSampler
from ogb.nodeproppred import PygNodePropPredDataset

In [ ]:
from ogb.nodeproppred import PygNodePropPredDataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = PygNodePropPredDataset(name = "ogbn-products")
dataset = TensorDataset(torch.tensor(dataset[0].x).to(torch.device(device)), torch.tensor(dataset[0].y).to(torch.device(device)))

In [15]:
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
batch_size = 1024

generator1 = torch.Generator().manual_seed(42)
_, _, test_set = torch.utils.data.random_split(dataset, [train_size, val_size, test_size], generator=generator1)

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

NameError: name 'dataset' is not defined

In [5]:
bow_model = torch.load('../models/bow-model-checkpoint-100.pt', map_location=torch.device('cpu'))


In [13]:
def evaluate_model(model, loader):
    model.eval()
    y_true = []
    y_pred = []
    for data in loader:
        x, y = data
        y_true.append(y)
        y_pred.append(torch.argmax(model(x), dim=1))
    y_true = torch.cat(y_true, dim=0)
    y_pred = torch.cat(y_pred, dim=0)
    return metrics.f1_score(y_true.cpu().numpy(), y_pred.cpu().numpy(), average='weighted')

In [14]:
evaluate_model(bow_model, test_loader)

NameError: name 'test_loader' is not defined